In [ ]:
# import os
# from google.colab import drive
# drive.mount('/content/gdrive/')
# print(os.getcwd())
# os.chdir("/content/gdrive/MyDrive/Thrun lab/rebuttal Bandit MIPS")
# print(os.getcwd())
# !ls

!pip install -U transformers datasets evaluate accelerate
!nvidia-smi

example_text = "Maximum Inner Product Search (MIPS) is a ubiquitous task in " \
+ "machine learning applications such as recommendation systems. Given a query " \
+ "vector and n atom vectors in d-dimensional space, the goal of MIPS is to " \
+ "find the atom that has the highest inner product with the query vector"

from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-6.7b")
model = AutoModelForCausalLM.from_pretrained("facebook/opt-6.7b")


import torch

print(model.lm_head.weight.size())
norm = torch.norm(model.lm_head.weight, dim=1)
print(f"max: {norm.max()}/ min: {norm.min()}")

import os

def generate_queries():
  input_ids = tokenizer.encode(
            example_text,
            return_tensors="pt",
        )  # Encode input sequence
  output = model.model(input_ids=input_ids)

  data_dir = "mips_opt"
  os.makedirs(data_dir, exist_ok=True)
  torch.save(model.lm_head.weight, os.path.join(data_dir, "OPT_LM_HEAD_WEIGHT.pt"))
  torch.save(output.last_hidden_state[0,-10:, :], os.path.join(data_dir,"OPT_QUERIES.pt"))
  print(output.last_hidden_state[0,-10:, :].shape)

generate_queries()
